In [1]:
#python3 t2_split.py

In [2]:
import os, gc, subprocess, time, sys, shutil
from glob import glob
import scipy
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from tqdm import tqdm
#from tqdm.notebook import tqdm
import skimage.io
from skimage.transform import resize, rescale
from math import sqrt
import statistics

sys.path.append("..")

In [3]:
import PIL.Image
#PIL.Image.MAX_IMAGE_PIXELS = 400000000
PIL.Image.MAX_IMAGE_PIXELS = None

In [4]:
from panda_bvv_config import *

## 1. Inputs ##

In [5]:
Zoom = 2

#these 2 values are for x5 cross-fold validation:
# choices_=[0, 1, 2, 3, 4, 5]
# cfv_const= -2123
# last_train_label = 7500
# last_valid_label = 9400


In [6]:
#data-from folder:
new_proc_folder = os.path.join(base_path, 'gs_proc_inv')

## 2. Standard functions ##

## 5. Distribute processed files among training and validation folders ##

In [7]:
try:
    shutil.rmtree(train_cnn)
    shutil.rmtree(valid_cnn)
except OSError as ee:
    print ("Error: %s: " % ee)
os.mkdir(train_cnn)
os.mkdir(valid_cnn)
for grade in gs_classes:
    os.mkdir(os.path.join(train_cnn, f'gs{grade}'))
    os.mkdir(os.path.join(valid_cnn, f'gs{grade}'))
# 

In [11]:
def new_split_func(filenames, to_folder):
    
#split train files among ISUP categories
    with(tqdm(total=len(filenames))) as pbar:
        for filename in filenames:
            path_to_class = os.path.join(to_folder, filename.split('/')[-1])
            bashCommand = "cp " + filename + ' ' + path_to_class
            process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
            output, error = process.communicate()
            if error: print(str(error))
            pbar.update(1)

In [12]:
from sklearn.model_selection import train_test_split
for gsfolder in os.listdir(new_proc_folder):
    labeled_files = glob(new_proc_folder + '/'+ gsfolder + '/*.png')
    train_files, val_files = train_test_split(labeled_files, test_size=0.1, random_state=101010)
    gsout = os.path.join(train_cnn, gsfolder)
    new_split_func(train_files, gsout)
    gsout = os.path.join(valid_cnn, gsfolder)
    new_split_func(val_files, gsout)
    

100%|██████████| 102/102 [00:14<00:00,  7.23it/s]


In [15]:
import os
module_name = 't2_split_gs'

os.system('jupyter nbconvert --to python ' + module_name + '.ipynb')
with open(module_name + '.py', 'r') as f:
    lines = f.readlines()
with open(module_name + '.py', 'w') as f:
    for line in lines:
        if 'nbconvert --to python' in line:
            break
        else:
            f.write(line)